In [1]:
import os
import glob
import pandas as pd

In [2]:
products = ['사과', '감귤', '수박', '배', '포도', '양파', '대파', '오이', '상추', '고구마']

In [3]:
column_to_check = "PDLT_NM" 

# 품목 찾아서 월별 평균 가격 구하기
def cal_avg_price(product, df, column_name=column_to_check):
    # 해당 컬럼이 df에 있는지 확인
    if column_name in df.columns:
        # str.contains 사용 전에 해당 컬럼이 문자열인지 확인
        if df[column_name].dtype == 'object':
            filtered_df = df[df[column_name].str.contains(product, na=False)]
            if not filtered_df.empty:

                # AUC_SFBD_TOT_AMNT_PER_KG 칼럼의 평균 계산
                if 'AUC_SFBD_TOT_AMNT_PER_KG' in filtered_df.columns:
                    return filtered_df['AUC_SFBD_TOT_AMNT_PER_KG'].mean()
                else:
                    return None  # AUC_SFBD_TOT_AMNT_PER_KG 칼럼이 없는 경우

In [5]:
import re

folder_path = './2023-24-도매시세' 

# 폴더 내 csv 파일 경로 리스트 생성
csv_file_list = glob.glob(os.path.join(folder_path, '*.csv'))

# 품목별로 처리
for product in products:
    print(f"'{product}' 데이터 처리 중...")
    
    # 각 품목마다 all_data 초기화
    product_year_price_df = {
    'MONTH': [],
    'YEAR': [],
    'AVG_PRICE': [], # 월별 평균 가격을 저장할 리스트
    'PRODUCT': [] # 품목명을 저장할 리스트
    }

    product_year_price_df = pd.DataFrame(product_year_price_df)
    
    # 모든 파일을 순회하며 해당 품목 데이터 찾기
    for file_path in csv_file_list:
        # 파일 읽기
        df = pd.read_csv(file_path, encoding='euc-kr', low_memory=False)

        file_name = os.path.basename(file_path)

        # 파일명에서 연도와 월 추출 (파일명 패턴에 따라 수정 필요)
        # 예: "2023년 06월 공영도매시장 농협 공판장 품목산지별 물량 및 금액-202306.csv"
        match = re.search(r'(\d{4})년\s+(\d{2})월', file_name)
        if match:
            year = match.group(1)
            month = match.group(2)
        else:
            # 파일명 끝부분에서 추출 시도
            date_part = file_name.split('-')[-1].split('.')[0]  # -202306.csv에서 202306 추출
            if len(date_part) >= 6 and date_part.isdigit():
                year = date_part[:4]
                month = date_part[4:6]
            else:
                print(f"파일 {file_name}에서 연도와 월을 추출할 수 없습니다. 건너뜁니다.")
                continue
        
        # 'CHG_TOT_VOLM' 컬럼이 있는지 확인하고 이름 변경
        if 'CHG_TOT_VOLM' in df.columns:
            df.rename(columns={'CHG_TOT_VOLM': 'KG_CHG_TOT_VOLM'}, inplace=True)
        
        # 1KG 당 거래액 컬럼 추가
        df['AUC_SFBD_TOT_AMNT_PER_KG'] = df['AUC_SFBD_TOT_AMNT'] / df['KG_CHG_TOT_VOLM']
        
        # 인덱스 리셋
        df.reset_index(drop=True, inplace=True)
        
        # 현재 품목에 대한 데이터 찾기
        avg_price = cal_avg_price(product, df)
        new_row_dict = {'MONTH': month , 'YEAR': year, 'AVG_PRICE': avg_price, 'PRODUCT': product}
        product_year_price_df = pd.concat([product_year_price_df, pd.DataFrame([new_row_dict])], ignore_index=True)
    
    # 결과 확인 및 저장
    if product_year_price_df.empty:
        print(f"'{product}'에 대한 데이터가 없습니다.")
    else:
        # 파일명에 품목 이름 포함
        file_name = f"{product}_monthly_price_avg_data.csv"
        file_path = os.path.join("품목별-월평균-가격-데이터", file_name)
        
        # 결과 저장
        product_year_price_df.to_csv(file_path, index=False, encoding='euc-kr')
        print(f"'{product}' 데이터 {len(product_year_price_df)}행이 {file_name}으로 저장되었습니다.")

print("모든 품목 처리 완료!")

'사과' 데이터 처리 중...
'사과' 데이터 12행이 사과_monthly_price_avg_data.csv으로 저장되었습니다.
'감귤' 데이터 처리 중...
'감귤' 데이터 12행이 감귤_monthly_price_avg_data.csv으로 저장되었습니다.
'수박' 데이터 처리 중...
'수박' 데이터 12행이 수박_monthly_price_avg_data.csv으로 저장되었습니다.
'배' 데이터 처리 중...
'배' 데이터 12행이 배_monthly_price_avg_data.csv으로 저장되었습니다.
'포도' 데이터 처리 중...
'포도' 데이터 12행이 포도_monthly_price_avg_data.csv으로 저장되었습니다.
'양파' 데이터 처리 중...
'양파' 데이터 12행이 양파_monthly_price_avg_data.csv으로 저장되었습니다.
'대파' 데이터 처리 중...
'대파' 데이터 12행이 대파_monthly_price_avg_data.csv으로 저장되었습니다.
'오이' 데이터 처리 중...
'오이' 데이터 12행이 오이_monthly_price_avg_data.csv으로 저장되었습니다.
'상추' 데이터 처리 중...
'상추' 데이터 12행이 상추_monthly_price_avg_data.csv으로 저장되었습니다.
'고구마' 데이터 처리 중...
'고구마' 데이터 12행이 고구마_monthly_price_avg_data.csv으로 저장되었습니다.
모든 품목 처리 완료!
